<a href="https://colab.research.google.com/github/nvrancovich/gh22-twitter-PLN/blob/main/analisis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [21]:
import warnings
warnings.simplefilter(action='ignore')

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [8]:
pd.set_option('display.max_columns', None)

In [6]:
df = pd.read_csv('/content/gdrive/MyDrive/proyectos/futbol/partidos_csv',)

In [9]:
df.sample(5)

,semana,dia,fecha,hora,local,visitante,sedes,arbitro,temporada,resultado,local_posesion,visitante_posesion,local_goles,visitante_goles,local_asistencias,local_goles_penal,local_penales,local_disparos,local_disparos_puerta,local_amarillas,local_rojas,local_faltas_cometidas,local_faltas_recibidas,local_posiciones_adelantedas,local_pases_cruzados,local_derribos,local_intercepciones,local_goles_contra,visitante_asistencias,visitante_goles_penal,visitante_penales,visitante_disparos,visitante_disparos_puerta,visitante_amarillas,visitante_rojas,visitante_faltas_cometidas,visitante_faltas_recibidas,visitante_posiciones_adelantedas,visitante_pases_cruzados,visitante_derribos,visitante_intercepciones,visitante_goles_contra,local_puntos_acumulados_temporada,visitante_puntos_acumulados_temporada
1357,18,Jue,2020-01-30,21:10,Vélez Sarsf,Aldosivi,Estadio José Amalfitani,Hernán Mastrángelo,2019-2020,empate,0.51,0.49,1,1,1,0,0,10,5,2,1,11,22,0,17,9,7,0,1,0,0,5,1,5,0,26,11,1,10,16,12,0,27,15
1448,2,Dom,2021-07-25,13:30,Patronato,Sarmiento,Estadio Presbítero Bartolomé Grella,Andrés Merlos,2021,local,0.38,0.62,2,0,2,0,0,12,3,4,0,14,12,2,9,7,12,0,0,0,0,12,4,2,0,12,14,2,32,17,6,0,3,0
1801,4,Dom,2022-06-19,18:00,CA Unión,River Plate,Estadio 15 de Abril,Pablo Echavarría,2022,visitante,0.53,0.47,1,5,1,0,0,12,4,2,0,8,16,1,26,10,8,0,4,0,0,13,7,3,0,16,8,0,12,13,16,0,4,2
935,9,Dom,2018-10-21,20:00,San Martin,Racing Club,Estadio Eva Perón de San Miguel de Tucum...,Nicolás Lamolina,2018-2019,local,0.32,0.68,2,1,2,0,0,8,4,5,1,11,11,1,18,13,15,0,1,0,0,18,9,4,0,14,10,2,28,14,9,0,4,20
349,24,Dom,2017-05-14,14:00,Colón,CA Unión,Estadio Brigadier General Estanislao Lóp...,Darío Herrera,2016-2017,local,0.61,0.39,1,0,0,1,1,15,1,2,0,18,22,2,36,7,13,1,0,0,0,5,2,5,0,23,18,0,8,12,6,0,43,28


In [56]:
df_arbitros = df[['arbitro','local_amarillas','visitante_amarillas', 'local_rojas', 'visitante_rojas', 'local_faltas_cometidas', 'visitante_faltas_cometidas','local_faltas_recibidas','visitante_faltas_recibidas']]
df_arbitros['amarillas'] = df_arbitros['local_amarillas'] + df_arbitros['visitante_amarillas']
df_arbitros['rojas'] = df_arbitros['local_rojas'] + df_arbitros['visitante_rojas']
df_arbitros['faltas'] = df_arbitros['local_faltas_cometidas'] + df_arbitros['visitante_faltas_cometidas']
df_arbitros = df_arbitros[['arbitro', 'amarillas', 'rojas', 'faltas']]
df_arbitros = df_arbitros.groupby('arbitro').describe()[df_arbitros.groupby('arbitro').count()['amarillas'] > 10].reset_index()
df_arbitros

arbitro amarillas                                           \
                           count      mean       std  min   25%  50%  75%   
0        Andrés Merlos     123.0  4.235772  2.135114  0.0  3.00  4.0  6.0   
1          Ariel Penel      99.0  4.363636  1.992099  1.0  3.00  4.0  6.0   
2        Darío Herrera     127.0  5.425197  2.173139  1.0  4.00  5.0  7.0   
3           Diego Abal      98.0  4.418367  1.804482  0.0  3.00  4.0  5.0   
4        Facundo Tello     110.0  5.600000  2.081813  1.0  4.00  5.0  7.0   
5     Federico Beligoy      35.0  5.200000  1.778962  1.0  4.00  5.0  6.0   
6   Fernando Echenique     102.0  4.401961  1.884065  0.0  3.00  4.0  5.0   
7    Fernando Espinoza     120.0  5.008333  2.295395  0.0  3.00  5.0  6.0   
8   Fernando Rapallini     113.0  6.035398  2.337311  2.0  4.00  6.0  8.0   
9       Germán Delfino      94.0  5.074468  2.195985  0.0  4.00  5.0  7.0   
10  Hernán Mastrángelo      87.0  4.919540  2.075429  0.0  4.00  5.0  6.0   
11      Héctor Paletta      13.0  5.076923  1.656379  2.0  4.00  5.0  6.0   
12        Jorge Baliño     104.0  4.721154  1.813952  1.0  3.75  5.0  6.0   
13         Juan Pompei      37.0  5.000000  1.885618  1.0  4.00  5.0  7.0   
14  Leandro Rey Hilfer      33.0  4.606061  2.030077  0.0  3.00  5.0  6.0   
15    Mariano González      15.0  4.533333  2.325838  1.0  3.00  4.0  6.0   
16      Mauro Vigliano      78.0  4.461538  1.958613  0.0  3.00  4.0  6.0   
17      Nazareno Arasa      45.0  5.177778  1.774682  2.0  4.00  5.0  6.0   
18    Nicolás Lamolina      98.0  5.602041  2.133386  1.0  4.00  6.0  8.0   
19       Néstor Pitana      86.0  5.279070  2.423664  0.0  3.00  5.0  7.0   
20        Pablo Dóvalo      42.0  4.642857  2.009992  0.0  4.00  5.0  6.0   
21    Pablo Echavarría      95.0  4.684211  2.179514  0.0  3.00  5.0  6.0   
22    Patricio Loustau     122.0  5.409836  2.521819  0.0  4.00  5.0  7.0   
23     Pedro Argañaraz      19.0  4.263158  1.726979  1.0  3.00  4.0  5.0   
24       Silvio Trucco     120.0  4.991667  2.076260  0.0  3.00  5.0  6.0   
25         Yael Falcón      39.0  5.846154  2.058851  0.0  5.00  6.0  7.0   

          rojas                                               faltas  \
     max  count      mean       std  min  25%  50%   75%  max  count   
0   15.0  123.0  0.268293  0.529044  0.0  0.0  0.0  0.00  2.0  123.0   
1   11.0   99.0  0.242424  0.516877  0.0  0.0  0.0  0.00  3.0   99.0   
2   11.0  127.0  0.401575  0.633206  0.0  0.0  0.0  1.00  2.0  127.0   
3    9.0   98.0  0.244898  0.498683  0.0  0.0  0.0  0.00  2.0   98.0   
4   12.0  110.0  0.390909  0.678577  0.0  0.0  0.0  1.00  3.0  110.0   
5    9.0   35.0  0.342857  0.591253  0.0  0.0  0.0  1.00  2.0   35.0   
6   10.0  102.0  0.196078  0.445866  0.0  0.0  0.0  0.00  2.0  102.0   
7   13.0  120.0  0.291667  0.627062  0.0  0.0  0.0  0.00  3.0  120.0   
8   14.0  113.0  0.327434  0.712672  0.0  0.0  0.0  0.00  4.0  113.0   
9   10.0   94.0  0.436170  0.711783  0.0  0.0  0.0  1.00  3.0   94.0   
10  10.0   87.0  0.264368  0.516242  0.0  0.0  0.0  0.00  2.0   87.0   
11   8.0   13.0  0.230769  0.438529  0.0  0.0  0.0  0.00  1.0   13.0   
12  10.0  104.0  0.288462  0.496064  0.0  0.0  0.0  1.00  2.0  104.0   
13   8.0   37.0  0.216216  0.534121  0.0  0.0  0.0  0.00  2.0   37.0   
14   9.0   33.0  0.181818  0.464660  0.0  0.0  0.0  0.00  2.0   33.0   
15   9.0   15.0  0.133333  0.351866  0.0  0.0  0.0  0.00  1.0   15.0   
16  10.0   78.0  0.179487  0.418519  0.0  0.0  0.0  0.00  2.0   78.0   
17  11.0   45.0  0.222222  0.420437  0.0  0.0  0.0  0.00  1.0   45.0   
18   9.0   98.0  0.387755  0.667858  0.0  0.0  0.0  1.00  3.0   98.0   
19  11.0   86.0  0.313953  0.579282  0.0  0.0  0.0  0.75  2.0   86.0   
20   9.0   42.0  0.333333  0.611542  0.0  0.0  0.0  0.75  2.0   42.0   
21  10.0   95.0  0.200000  0.451946  0.0  0.0  0.0  0.00  2.0   95.0   
22  11.0  122.0  0.516393  0.864675  0.0  0.0  0.0  1.00  4.0  122.0   
23   9.0   19.0  0.210526  0.418854  0.0  0.0  0.0  0.00  

In [57]:
px.scatter( x=df_arbitros['amarillas']['mean'], y=df_arbitros['rojas']['mean'], hover_name=df_arbitros['arbitro'])